In [2]:
import os
import requests
import pandas as pd
import json

In [18]:
# DOE LEAD API
DOE_LEAD_URL = "https://lead.openei.org/api/geographies"

# Outputs directory
outdir_json = "outputs/JSON"
outdir_csv = "outputs/CSV"

In [9]:
# read all Michigan census tracts
mi_tract_f = open("data/mi_census_tracts.csv", "r")
mi_tracts_raw = mi_tract_f.readlines()
mi_tract_f.close()

# go through each record and append MI census tract to a list
mi_tracts = []
for row in mi_tracts_raw[1:]:
   row = row.rstrip("\n")
   data = row.split(",")
   geoid = data[0]
   county = data[1]
   state = data[2]

   mi_tracts.append(geoid)
# Create a string comma separated (no whitespace) of MI census tracts
mi_tracts_str = (",").join(mi_tracts)

In [12]:
mi_tracts_str

In [ ]:
# Request average energy burden across all AMI groups
resp = requests.post(
   DOE_LEAD_URL,
   json = {
      "type": "census-tracts", # Make sure to include type: "census-tracts" in post requests to get data by census tracts
      "geoids": mi_tracts_str, # all MI census tracts
      "ami": ["0-30", "30-60", "60-80", "80-100", "100+"] # Data will be aggregated across all AMI groups requested
   }
)

# Get JSON data from HTTP response
data = resp.json()

In [16]:
# Serialize JSON data as a string
data_serialized = json.dumps(data, indent = 3)
# Write out JSON data
out_f = open(f'{outdir_json}/mi_all_ami.json', "w")
out_f.write(data_serialized)
out_f.close()

In [ ]:
# All possible AMI groups
ami = ["0-30", "30-60", "60-80", "80-100", "100+"]
# Request energy burden data per group
for ami_grp in ami:
   # HTTP post request for energy burden across all MI census tracts for a specific AMI group
   resp = requests.post(
      DOE_LEAD_URL,
      json = {
         "type": "census-tracts", # Make sure to include type: "census-tracts" in post requests to get data by census tracts
         "geoids": mi_tracts_str, # This should be a string of census tract geoids comma separated no whitespace
         "ami": [ami_grp] # Data will be aggregated across all AMI groups requested
      }
   )

   print(f'{ami_grp} {resp.status_code}')
   # Get JSON data from HTTP response
   data = resp.json()
   # Serialize JSON data
   data_serialized = json.dumps(data, indent = 3)
   # Write JSON data out
   out_f = open(f'{outputs}/mi_ami_{ami_grp}.json', "w")
   out_f.write(data_serialized)
   out_f.close()


In [22]:
# Clean JSON data and structure as CSV
json_files = os.listdir(outdir_json)

for in_file in json_files:
   print(f'{outdir_json}/{in_file}')
   f = open(f'{outdir_json}/{in_file}')
   data = f.read()
   f.close()

   data = json.loads(data)
   total_items = data['total_items']
   items = data['items']

   df = pd.DataFrame.from_dict(items)
   outfilename = f'{in_file[:-5]}.csv'
   print(f'Writing {os.path.join(outdir_csv, outfilename)}')
   df.to_csv(os.path.join(outdir_csv, outfilename), index=False)
print("DONE")

outputs/JSON/mi_ami_30-60.json
Writing outputs/CSV/mi_ami_30-60.csv
outputs/JSON/mi_ami_80-100.json
Writing outputs/CSV/mi_ami_80-100.csv
outputs/JSON/mi_ami_100+.json
Writing outputs/CSV/mi_ami_100+.csv
outputs/JSON/mi_ami_60-80.json
Writing outputs/CSV/mi_ami_60-80.csv
outputs/JSON/mi_ami_0-30.json
Writing outputs/CSV/mi_ami_0-30.csv
DONE
